In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import signal
from sklearn.metrics import mean_squared_error

import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(42)
%load_ext autoreload
%autoreload 2

In [ ]:
from LSTM_functions import split_and_reshape_data, fit_sequential_LSTM, get_LSTM_predictions,\
inverse_transform, calculate_RMSE, prep_predictions_for_plotting, plot_data_LSTM_predictions

In [ ]:
from timeseries_functions import index_to_datetime, plot_all_df_columns, weekly_resample, plot_series,\
plot_series_save_fig, plot_series_and_differences, run_augmented_Dickey_Fuller_test, \
plot_autocorrelation, plot_partial_autocorrelation, plot_decomposition

In [ ]:
from timeseries_functions import make_col_vector, make_design_matrix, fit_linear_trend,\
plot_trend_data, plot_linear_trend

In [ ]:
# plt.rcParams.keys()

### import data

In [ ]:
dr_df = pd.read_csv('doctors_hours_per_provider.csv', index_col=0)
RNPA_df = pd.read_csv('RNPA_hours_per_provider.csv', index_col=0)
ther_df = pd.read_csv('therapists_hours_per_provider.csv', index_col=0)

In [ ]:
dr_df.index

In [ ]:
all_df = [dr_df, RNPA_df, ther_df]

In [ ]:
# convert index to datetime
for df in all_df:
    index_to_datetime(df)

In [ ]:
train_start = '2015-01-12'
train_end = '2018-02-26'
test_start = '2018-03-05'

In [ ]:
dr_train = dr_df.loc[train_start:train_end]
dr_test = dr_df.loc[test_start:]

In [ ]:
RNPA_train = RNPA_df.loc[train_start:train_end]
RNPA_test = RNPA_df.loc[test_start:]

In [ ]:
ther_train = ther_df.loc[train_start:train_end]
ther_test = ther_df.loc[test_start:]

In [ ]:
train_df = [dr_train, RNPA_train, ther_train]
test_df = [dr_test, RNPA_test, ther_test]

In [ ]:
dr_train

### visualize major holidays and their effects on series

additional dips seen around May-June, September

In [ ]:
# try with just Dr data
new_years = ['2016-01-01', '2017-01-01', '2018-01-01']
fourth_of_july = ['2015-07-04', '2016-07-04', '2017-07-04']
Christmas = ['2015-12-15', '2016-12-25', '2017-12-25']
Thanksgiving = ['2015-11-26', '2016-11-24', '2017-11-23']
holidays = ['2016-01-01', '2017-01-01', '2018-01-01','2015-07-04', '2016-07-04', '2017-07-04',\
            '2015-12-15', '2016-12-25', '2017-12-25', '2015-11-26', '2016-11-24', '2017-11-23']

fig = plot_series_save_fig(dr_df['Hours'], xlabel='Date', ylabel='Hours per Week', plot_name='Doctors',\
            v_lines=holidays, figsize=(12,6), figname="dr_data_w_holidays.png")

In [ ]:
plot_series_save_fig(RNPA_df['Hours'], xlabel='Date', ylabel='Hours per Week', plot_name='RN/PA Hourly with Holidays (red)',\
            v_lines=holidays, figsize=(12,6), figname="RN_PA_data_w_holidays.pdf")

In [ ]:
plot_series_save_fig(ther_df['Hours'], xlabel='Date', ylabel='Hours per Week', plot_name='Therapists Hourly with Holidays (red)',\
            v_lines=holidays, figsize=(12,6), figname="therapist_data_w_holidays.pdf")

### Long Short-Term Memory network (LSTM)

In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
np.random.seed(42)

In [ ]:
dr_df['Hours'].shape, len(dr_train), len(dr_test), 9/164
# use test set = 0.05 to be consistent with other models (test = 3/18-4/18)

In [ ]:
data = dr_df['Hours']
# reshape to 2D array
data = data.reshape(-1,1)
# scale/normalize data
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)

In [ ]:
# split data into training and test sets
# train_size = int(len(data) * 0.67)
train_size = int(len(data) * 0.95) # to be consistent with train/test split other models
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [ ]:
# convert data values into dataset matrix
def create_dataset(data, num_steps=12):
    dataX, dataY = [], []
    for i in range(len(data)-num_steps-1):
        a = data[i:(i+num_steps),0]
        dataX.append(a)
        dataY.append(data[i + num_steps, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
num_steps=1
trainX, trainY = create_dataset(train, num_steps)
testX, testY = create_dataset(test, num_steps)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
testX.shape, trainX.shape, testY.shape, trainY.shape

In [ ]:
# create and fit LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, num_steps))) # 4 neurons in first hidden layer
model.add(Dense(1))     # 1 neuron in output layer
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
# testX

In [ ]:
# get predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# testPredict

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
trainY.shape, testY.shape

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
train.shape, test.shape

In [ ]:
#shift train and test predictions for plotting
trainPredictPlot = np.empty_like(data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[num_steps:len(trainPredict)+num_steps, :] = trainPredict
testPredictPlot = np.empty_like(data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(num_steps*2)+1:len(data)-1, :] = testPredict

In [ ]:
# save predictions in dataframe/table format

In [ ]:
# plot baseline and predictions
params = {'figure.figsize': [12,8],'axes.grid': False,'axes.grid.axis': 'both', 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
plt.plot(scaler.inverse_transform(data), label='actual')
plt.plot(trainPredictPlot, linestyle='--',  label='predicted')
plt.plot(testPredictPlot, linestyle='--', label='predicted')
plt.xlabel('Number of Weeks')
plt.ylabel('Hours per Week')
plt.title('Doctors LSTM Model')
# plt.legend()
plt.savefig('dr_LSTM.png')
plt.show()

In [ ]:
def split_and_reshape_data(data, split_at=0.67, num_steps=1):
    train_size = int(len(data) * split_at)
    test_size = len(data) - train_size
    train, test = data[0:train_size,:], data[train_size:len(data),:]
    train, test = data[0:train_size,:], data[train_size:len(data),:]
    trainX, trainY = create_dataset(train, num_steps)
    testX, testY = create_dataset(test, num_steps)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    return trainX, trainY, testX, testY

In [ ]:
def fit_sequential_LSTM(trainX, trainY, add_layers=4, input_shape=(1,1),\
                        density=1, epochs=100, batch_size=1, optimizer='adam', verbose=2, \
                        loss='mean_squared_error'):
    model = Sequential()
    model.add(LSTM(add_layers, input_shape=input_shape))
    model.add(Dense(density))
    model.compile(loss=loss, optimizer=optimizer)
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)

In [ ]:
def get_LSTM_predictions(trainX, testX):
    "Get predictions for training and test data"
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    return trainPredict, testPredict

def inverse_transform(trainY, testY, trainPredict, testPredict):  
    "Inverse transform train and test set pedictions"
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    return trainY, testY, trainPredict, testPredict

In [ ]:
def calculate_RMSE(trainY, testY, trainPredict, testPredict):
    "calculate root mean squared error for training and test predictions"
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
def prep_predictions_for_plotting(data, trainPredict, testPredict, num_steps=1):
    trainPredictPlot = np.empty_like(data)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[num_steps:len(trainPredict)+num_steps, :] = trainPredict
    testPredictPlot = np.empty_like(data)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(trainPredict)+(num_steps*2)+1:len(data)-1, :] = testPredict
    return trainPredictPlot, testPredictPlot

In [ ]:
def plot_data_LSTM_predictions(data, trainPredictPlot, testPredictPlot,\
                              title='', xlabel='', ylabel=''):
    fig = plt.figure()
    plt.plot(scaler.inverse_transform(data), label='actual')
    plt.plot(trainPredictPlot, linestyle='--',  label='predicted')
    plt.plot(testPredictPlot, linestyle='--', label='predicted')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()
    return 

### for RN/PA category

In [ ]:
data = RNPA_df['Hours']

In [ ]:
# reshape to 2D array
data = data.reshape(-1,1)
# scale/normalize data
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)

In [ ]:
trainX, trainY, testX, testY = split_and_reshape_data(data, split_at=0.95, num_steps=1)

fit_sequential_LSTM(trainX, trainY, add_layers=4, input_shape=(1,1),\
                        density=1, epochs=100, batch_size=1, optimizer='adam', verbose=2, \
                        loss='mean_squared_error')

In [ ]:
# save predictions in dataframe/table format

In [ ]:
trainPredict, testPredict = get_LSTM_predictions(trainX, testX)

trainY, testY, trainPredict, testPredict = inverse_transform(trainY, testY, trainPredict, testPredict)

calculate_RMSE(trainY, testY, trainPredict, testPredict)

trainPredictPlot, testPredictPlot = prep_predictions_for_plotting(data, trainPredict, testPredict, num_steps=1)

plot_data_LSTM_predictions(data, trainPredictPlot, testPredictPlot,\
   title='RN/PA LSTM Model', xlabel='Number of Weeks', ylabel='Hours per Week')

### Therapists

In [ ]:
data = ther_df['Hours']
# reshape to 2D array
data = data.reshape(-1,1)
# scale/normalize data
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)

In [ ]:
trainX, trainY, testX, testY = split_and_reshape_data(data, split_at=0.95, num_steps=1)

fit_sequential_LSTM(trainX, trainY, add_layers=4, input_shape=(1,1),\
                        density=1, epochs=100, batch_size=1, optimizer='adam', verbose=2, \
                        loss='mean_squared_error')

In [ ]:
trainPredict, testPredict = get_LSTM_predictions(trainX, testX)

trainY, testY, trainPredict, testPredict = inverse_transform(trainY, testY, trainPredict, testPredict)

calculate_RMSE(trainY, testY, trainPredict, testPredict)

trainPredictPlot, testPredictPlot = prep_predictions_for_plotting(data, trainPredict, testPredict, num_steps=1)

plot_data_LSTM_predictions(data, trainPredictPlot, testPredictPlot,\
    title='Therapists LSTM Model', xlabel='Number of Weeks', ylabel='Hours per Week')

#### Future predictions

In [ ]:
# reshape and scale the data
data = dr_df['Hours'].copy()
# reshape to 2D array
data = data.reshape(-1,1)
# scale/normalize data
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)

In [ ]:
# convert data values into dataset matrix
def create_dataset(data, num_steps=12):
    dataX, dataY = [], []
    for i in range(len(data)-num_steps-1):
        a = data[i:(i+num_steps),0]
        dataX.append(a)
        dataY.append(data[i + num_steps, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_entire_dataset(data, num_steps=12):
    X = []
    for i in range(len(data)-num_steps-1):
        a = data[i:(i+num_steps),0]
        X.append(a)
    return np.array(X)

In [ ]:
data.shape

In [ ]:
dataset = create_entire_dataset(data, num_steps=12)

In [ ]:
dataset.shape

In [ ]:
dataset = dataset.reshape(dataset.shape[0], 12, 1)

In [ ]:
dataset.shape

In [ ]:
X = dataset[:, :-1, :]
y = dataset[:, 1:, :]

In [ ]:
X.shape, y.shape

In [ ]:
# design network
model = Sequential()
model.add(LSTM(4, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# fit network
model.fit(X, y, epochs=100, batch_size=1, verbose=2)

#### multivariate LSTM

In [ ]:
all_df = [dr_df, RNPA_df, ther_df]

In [ ]:
dr_df.columns, dr_df.index

In [ ]:
def plot_all_df_columns(df, col_nums, params, title='', xlabel=''):
    i = 1
    values = df.values
    for col in col_nums:
        plt.subplot(len(col_nums), 1, i)
        plt.plot(values[:, col])
        plt.title(title)
        plt.ylabel(dr_df.columns[col])
        plt.xlabel(xlabel)
        i += 1
    plt.tight_layout()
    plt.show()

In [ ]:
col_nums = [0, 1, 2]
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Large', 'font.size': 12.0, \
'lines.linewidth': 3}

In [ ]:
plt.rcParams.update(params)
plot_all_df_columns(dr_df, col_nums, params, title='Doctors', xlabel='Time in Weeks')

In [ ]:
plot_all_df_columns(RNPA_df, col_nums, title='RN/PAs', xlabel='Time in Weeks')

In [ ]:
plot_all_df_columns(ther_df, col_nums, title='Therapists', xlabel='Time in Weeks')

In [ ]:
# all_df = [dr_df, RNPA_df, ther_df]S